## Using Ultralytics to import the Yolo V12. 

In [ ]:
from ultralytics import YOLO        # importing YOLO from ultralytics

model = YOLO("yolov12n.pt")     # Importing yolo v12n

print(model)        # prining model architecture to see which layers are backbone. 



YOLO(
  (model): DetectionModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(16, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=2, bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (2): C3k2(
        (cv1): Conv(
          (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (cv2): Conv(
          (conv): Conv2d(48, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, trac

### After layer 6 we can see the attention modules. So for fine tuning we just freeze till the layer 6 after that the attention (neck and head) modules are left trainable for the custom dataset. The basic feature extraction takes place in the backbone which we froze. This way the model adapts to new data while preserving foundational visual features.

In [5]:
for param in model.model.parameters():      # making all layers trainable first
    param.requires_grad = True


for idx in range(6):
    for param in model.model.model[idx].parameters():
        param.requires_grad = False     # Freezing the backbone of the architecture. after inspecting the model we can see the first 6 layers are responsible for the basic feature extration


for i, (name, param) in enumerate(model.model.named_parameters()):      # visualizing which layers are trainable
    print(f"{i:3d}: {'FROZEN' if not param.requires_grad else 'TRAINABLE'} - {name}")

  0: FROZEN - model.0.conv.weight
  1: FROZEN - model.0.bn.weight
  2: FROZEN - model.0.bn.bias
  3: FROZEN - model.1.conv.weight
  4: FROZEN - model.1.bn.weight
  5: FROZEN - model.1.bn.bias
  6: FROZEN - model.2.cv1.conv.weight
  7: FROZEN - model.2.cv1.bn.weight
  8: FROZEN - model.2.cv1.bn.bias
  9: FROZEN - model.2.cv2.conv.weight
 10: FROZEN - model.2.cv2.bn.weight
 11: FROZEN - model.2.cv2.bn.bias
 12: FROZEN - model.2.m.0.cv1.conv.weight
 13: FROZEN - model.2.m.0.cv1.bn.weight
 14: FROZEN - model.2.m.0.cv1.bn.bias
 15: FROZEN - model.2.m.0.cv2.conv.weight
 16: FROZEN - model.2.m.0.cv2.bn.weight
 17: FROZEN - model.2.m.0.cv2.bn.bias
 18: FROZEN - model.3.conv.weight
 19: FROZEN - model.3.bn.weight
 20: FROZEN - model.3.bn.bias
 21: FROZEN - model.4.cv1.conv.weight
 22: FROZEN - model.4.cv1.bn.weight
 23: FROZEN - model.4.cv1.bn.bias
 24: FROZEN - model.4.cv2.conv.weight
 25: FROZEN - model.4.cv2.bn.weight
 26: FROZEN - model.4.cv2.bn.bias
 27: FROZEN - model.4.m.0.cv1.conv.weigh

### Main training loop

In [ ]:
# Main training function

model.train(
    data='/home/adil/yolo_v12n/dataset_door_handle/data.yaml',
    epochs=100,
    imgsz=640,
    batch=16,
    lr0=0.001,
    project='yolov12_finetune',
    name='freeze_backbone'
)

New https://pypi.org/project/ultralytics/8.3.146 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.63 🚀 Python-3.11.11 torch-2.2.2+cu121 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 5815MiB)
engine/trainer: task=detect, mode=train, model=yolov12n.pt, data=/home/adil/yolo_v12n/dataset_door_handle/data.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=yolov12_finetune, name=freeze_backbone3, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=Fa

train: Scanning /home/adil/yolo_v12n/dataset_door_handle/train/labels.cache... 1441 images, 2 backgrounds, 0 corrupt: 100%|██████████| 1441/1441 [00:00<?, ?it/s]


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


/home/adil/anaconda3/envs/yolov12/lib/python3.11/site-packages/albumentations/__init__.py:28: UserWarning: A new version of Albumentations is available: '2.0.8' (you have '2.0.4'). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()
/home/adil/yolo_v12n/yolov12/ultralytics/data/augment.py:1853: UserWarning: Argument(s) 'quality_lower' are not valid for transform ImageCompression
  A.ImageCompression(quality_lower=75, p=0.0),
val: Scanning /home/adil/yolo_v12n/dataset_door_handle/valid/labels.cache... 141 images, 0 backgrounds, 0 corrupt: 100%|██████████| 141/141 [00:00<?, ?it/s]


Plotting labels to yolov12_finetune/freeze_backbone3/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.001' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 121 weight(decay=0.0), 128 weight(decay=0.0005), 127 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to yolov12_finetune/freeze_backbone3
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all        141        174       0.88      0.672      0.761      0.523



100 epochs completed in 0.409 hours.
Optimizer stripped from yolov12_finetune/freeze_backbone3/weights/last.pt, 5.4MB
Optimizer stripped from yolov12_finetune/freeze_backbone3/weights/best.pt, 5.4MB

Validating yolov12_finetune/freeze_backbone3/weights/best.pt...
Ultralytics 8.3.63 🚀 Python-3.11.11 torch-2.2.2+cu121 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 5815MiB)
YOLOv12n summary (fused): 376 layers, 2,508,539 parameters, 0 gradients, 5.8 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:00<00:00,  5.26it/s]


                   all        141        174       0.88      0.672      0.761      0.524
Speed: 0.2ms preprocess, 2.1ms inference, 0.0ms loss, 1.0ms postprocess per image
Results saved to yolov12_finetune/freeze_backbone3


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x776eed7c6110>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 

# Inference

In [ ]:
from ultralytics import YOLO

model = YOLO('yolov12_finetune/freeze_backbone3/weights/best.pt')

image_path = "dataset_door_handle/test/images/01a4ebfc00807e0e_jpg.rf.79d72597726732761c65133c731fa82b.jpg"     # path to image for inference


results = model(image_path, conf=0.75)      
results[0].show()
results[0].save(filename='output.jpg')


image 1/1 /home/adil/yolo_v12n/dataset_door_handle/test/images/01a4ebfc00807e0e_jpg.rf.79d72597726732761c65133c731fa82b.jpg: 640x640 1 handle, 10.6ms
Speed: 1.3ms preprocess, 10.6ms inference, 0.8ms postprocess per image at shape (1, 3, 640, 640)


'output.jpg'